In [4]:
%reload_ext autoreload
%autoreload 2

In [1]:
import dill as pickle
import aesara
import aesara.tensor as aet
import pandas as pd

import pycmtensor as cmt
from pycmtensor.models import MNLogit
from pycmtensor.optimizers import Adam
from pycmtensor.expressions import Beta, Weights
from pycmtensor.results import Results, Predict

swissmetro = pd.read_csv("data/swissmetro.dat", sep="\t")
db = cmt.Database(name="swissmetro", pandasDatabase=swissmetro, choiceVar="CHOICE")
globals().update(db.variables)
# Removing some observations
exclude = ((PURPOSE != 1) * (PURPOSE != 3) + (CHOICE == 0)) > 0
db.remove(exclude)

# additional steps to format database
db.data["CHOICE"] -= 1 # set the first choice to 0
db.choices = sorted(db.data["CHOICE"].unique()) # save original choices
db.autoscale(variables=['TRAIN_CO', 'TRAIN_TT', 'CAR_CO', 'CAR_TT', 
    'SM_CO', 'SM_TT'], default=100., verbose=False)

# declare model params here
b_cost = Beta("b_cost", 0.0, None, None, 0)
b_time = Beta("b_time", 0.0, None, None, 0)
asc_train = Beta("asc_train", 0.0, None, None, 0)
asc_car = Beta("asc_car", 0.0, None, None, 0)
asc_sm = Beta("asc_sm", 0.0, None, None, 1)

U_1 = b_cost * db["TRAIN_CO"] + b_time * db["TRAIN_TT"] + asc_train
U_2 = b_cost * db["SM_CO"] + b_time * db["SM_TT"] + asc_sm
U_3 = b_cost * db["CAR_CO"] + b_time * db["CAR_TT"] + asc_car

# specify the utility function and the availability conditions
U = [U_1, U_2, U_3]
AV = [db["TRAIN_AV"], db["SM_AV"], db["CAR_AV"]]
        
mymodel = MNLogit(u=U, av=AV, database=db, name="mymodel")
mymodel.add_params(locals())

# train function
model = cmt.train(mymodel, database=db, optimizer=Adam, batch_size=128, lr_init=0.01, max_epoch=30, notebook=True)

with open("myModel.pkl", "rb") as f:
    model = pickle.load(f)

result = Results(model, db, show_weights=True)

mymodel


In [3]:
from aesara.tensor.random.utils import RandomStream

class ResLogitLayer:
    def __init__(self, input, w_in, w_out, activation_in=None, activation_out=None):
        assert w_in.shape[1].eval() == w_out.shape[0].eval()
        if isinstance(input, (list, tuple)):
            assert len(input) == w_in.shape[0].eval(), f"index.0 of w_in must be of the same length as input"
            input = aet.stacklists(input).flatten(2)

        assert isinstance(w_in, (Weights)), "w_in must be of type Weights"
        assert isinstance(w_out, (Weights)), "w_out must be of type Weights"
        self.w_in = w_in()
        self.w_out = w_out()
        if activation_in == None:
            activation_in = aet.sigmoid
        if activation_out == None:
            activation_out = aet.sigmoid
            
        h = activation_in(aet.dot(input.T, self.w_in))
        output = activation_out(aet.dot(h, self.w_out)).T
        self.input = input
        self.weights = [self.w_in, self.w_out]
        self.output = output + input

    
    def __repr__(self):
        return f"ResLogitLayer([{self.w_in.shape.eval()}, {self.w_out.shape.eval()}])"

In [5]:
import aesara.d3viz as d3v
d3v.d3viz(model.loglikelihood, 'viz/MNLogit.html')

aesara.printing.pydotprint(mymodel.cost, "viz/print.png")
from IPython.display import Image
Image("viz/print.png", width="80%")

In [6]:
from pycmtensor.results import Predict
from pycmtensor.pycmtensor import PyCMTensorModel
import dill as pickle

class MNLmodel(PyCMTensorModel):
    def __init__(self, db):
        super().__init__(db)

with open("myModel.pkl", "rb") as f:
    model = pickle.load(f)

Predict(model, db).choices()

,CHOICE
0,1
1,1
2,1
3,1
4,1
...,...
6763,1
6764,1
6765,1
6766,1


In [7]:
from tqdm import tqdm, trange
from time import sleep

pbar = tqdm(total=100)
for i in range(10):
    sleep(0.1)
    pbar.update(10)
pbar.close()

pbar.format_dict

100%|██████████| 100/100 [00:01<00:00, 90.43it/s]


{'n': 100,
 'total': 100,
 'elapsed': 1.113598346710205,
 'ncols': None,
 'nrows': None,
 'prefix': '',
 'ascii': False,
 'unit': 'it',
 'unit_scale': False,
 'rate': None,
 'bar_format': None,
 'postfix': None,
 'unit_divisor': 1000,
 'initial': 0,
 'colour': None}